In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
import regex as re

In [2]:
games = []

test_links = ['https://www.ign.com/games/batman-dark-tomorrow']

for url in test_links:

    infos = {}
    
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        infos['game'] = soup.find('h1', {'class': 'display-title'}).getText()
    except:
        infos['game'] = ''

    try:
        all_devs = soup.find('div', {'class': 'developers-info'}).findAll('a')
        devs = []
        for dev in all_devs:
            devs.append(dev.getText())
        infos['developers'] = devs
    except:
        infos['developers'] = ''
    
    try:
        all_plubs = soup.find('div', {'class': 'publishers-info'}).findAll('a')
        plubs = []
        for plub in all_plubs:
            plubs.append(plub.getText())
        infos['publishers'] = plubs
    except:
        infos['publishers'] = ''
        
    try:
        frans = soup.find('div', {'class': 'franchises-info'}).findAll('a')
        franchises = []
        for fran in frans:
            franchises.append(fran.getText())            
        infos['franchises'] = franchises
    except:
        infos['franchises'] = ''
        
    try:
        feats = soup.find('div', {'class': 'features-info'}).findAll('a')
        features = []
        for feat in feats:
            features.append(feat.getText())
        infos['features'] = features
    except:
        infos['features'] = ''

    try:
        date = soup.find('div', {'class': 'initial-release-info'}).findAll('div', {'class': 'jsx-1164031273'})[1].getText()
        infos['released_date'] = datetime.strptime(date, '%b %d, %Y')
    except:
        infos['released_date'] = ''
    
    try:
        all_plats = soup.find('div', {'class': 'platforms-info'}).findAll('title')
        platforms = []
        for plat in all_plats:
            platforms.append(plat.getText())
        infos['platform'] = platforms
    except:
        infos['platform'] = ''
    
    try:
        all_genres = soup.find('div', {'class': 'genres-info'}).findAll('a')
        genres = []
        for gens in all_genres:
            genres.append(gens.getText())
        infos['genres'] = genres
    except:
        infos['genres'] = ''
    
    try:
        infos['score'] = float(soup.find('figure', {'class': 'review-score'}).find('figcaption').getText())
    except:
        infos['score'] = ''
    
    try:
        infos['score_text'] = soup.find('div', {'class': 'score-text'}).getText()
    except:
        infos['score_text'] = ''
        
    try:
        infos['esrb'] = re.sub('ESRB: ', '', soup.find('svg', {'class': 'jsx-2211565535'}).getText())
    except:
        infos['esrb'] = ''
        
    try:
        infos['esrb_info'] = soup.findAll('div', {'class': 'content-rating-info'})[1].getText().split(', ')
    except:
        infos['esrb_info'] = ''

    games.append(infos)

In [3]:
games

[{'game': 'Batman: Dark Tomorrow',
  'developers': ['Kemco'],
  'publishers': ['Kemco'],
  'franchises': ['DC Comics', 'Batman'],
  'features': ['Memory Card',
   'Vibration',
   'DualShock',
   '480p',
   'Dolby Digital 5.1'],
  'released_date': '',
  'platform': ['GameCube', 'Xbox', 'PlayStation 2'],
  'genres': ['Action', 'Adventure'],
  'score': 3.5,
  'score_text': 'awful',
  'esrb': 'Loading',
  'esrb_info': ['Violence',
   'Animated Violence',
   'Mild Animated Violence',
   'Realistic Violence']}]

In [130]:
current_consoles = ['xbox-360', 'gba', 'pc']

current_list = []

x = 0
y = 7
while x != 4:
    for console_n in current_consoles:

        c = DesiredCapabilities.CHROME
        c["pageLoadStrategy"] = "none"
        browser = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=c)
        browser.get("https://www.ign.com/reviews/games/{}".format(console_n))

        time.sleep(5)

        browser.execute_script("window.stop();")

        print('Starting scrap of {}'.format(console_n))

        pre_load = browser.execute_script('return document.body.scrollHeight')
        after_load = 0    
        while pre_load != after_load:
            pre_load = browser.execute_script('return document.body.scrollHeight')
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(y)
            after_load = browser.execute_script('return document.body.scrollHeight')

        print('Exited the loop')

        a_list = browser.find_elements_by_class_name('item-body')
        for a in a_list:
            if a.find_element_by_class_name('jsx-1785568775').text == 'GAME':
                link = a.find_element_by_tag_name('a').get_attribute('href')
                if link not in current_list:
                    current_list.append(link)

        print('The last added link was: {}'.format(current_list[-1]))

        print('Current size of set: {}'.format(len(current_list)))

        browser.quit()
    print(x)
    x += 1
    y += 1


Starting scrap of xbox-360
Exited the loop
The last added link was: https://www.ign.com/games/max-payne
Current size of set: 1553

Starting scrap of gba
Exited the loop
The last added link was: https://www.ign.com/games/namco-museum-gba
Current size of set: 2027

Starting scrap of pc
Exited the loop


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=105.0.5195.125)


In [123]:
c = DesiredCapabilities.CHROME
c["pageLoadStrategy"] = "none"
browser = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=c)
browser.get("https://www.ign.com/reviews/games/xbox-360")

In [144]:
#last_height = browser.execute_script('return document.body.scrollHeight')

#while (datetime.now().min < 20):
timeout = time.time() + 60*5 # 5 minutes from now
while True:
    test = 0
    if test == 5 or time.time() > timeout:
        break
    test = test - 1
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(2)
    #pre_load = browser.execute_script('return document.body.scrollHeight')
    #after_load = 0
    #while pre_load != after_load:
    #    pre_load = browser.execute_script('return document.body.scrollHeight')
    
    #    after_load = browser.execute_script('return document.body.scrollHeight')

print('Exited the loop')


Exited the loop


In [145]:
a_list = browser.find_elements_by_class_name('item-body')
current_list = []
for a in a_list:
    if a.find_element_by_class_name('jsx-1785568775').text == 'GAME':
        link = a.find_element_by_tag_name('a').get_attribute('href')
        if link not in current_list:
            current_list.append(a.find_element_by_tag_name('a').get_attribute('href'))
print('The last added link was: {}'.format(links[-1]))

The last added link was: https://www.ign.com/games/killer-instinct


In [21]:
a_list = browser.find_elements_by_class_name('item-body')

for a in a_list:
    if a.find_element_by_class_name('jsx-1785568775').text == 'GAME':
        link = a.find_element_by_tag_name('a').get_attribute('href')
        if link not in links:
            links.append(a.find_element_by_tag_name('a').get_attribute('href'))

In [201]:
games = []

test_links = safeList[:5]

for url in test_links:

    infos = {}
    
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        infos['game'] = soup.find('h1', {'class': 'display-title'}).getText()
    except:
        infos['game'] = ''

    try:
        all_devs = soup.find('div', {'class': 'developers-info'}).findAll('a')
        devs = []
        for dev in all_devs:
            devs.append(dev.getText())
        infos['developers'] = devs
    except:
        infos['developers'] = ''
    
    try:
        all_plubs = soup.find('div', {'class': 'publishers-info'}).findAll('a')
        plubs = []
        for plub in all_plubs:
            plubs.append(plub.getText())
        infos['publishers'] = plubs
    except:
        infos['publishers'] = ''
        
    try:
        frans = soup.find('div', {'class': 'franchises-info'}).findAll('a')
        franchises = []
        for fran in frans:
            franchises.append(fran.getText())            
        infos['franchises'] = franchises
    except:
        infos['franchises'] = ''
        
    try:
        feats = soup.find('div', {'class': 'features-info'}).findAll('a')
        features = []
        for feat in feats:
            features.append(feat.getText())
        infos['features'] = features
    except:
        infos['features'] = ''

    try:
        date = soup.find('div', {'class': 'initial-release-info'}).findAll('div', {'class': 'jsx-1164031273'})[1].getText()
        infos['released_date'] = datetime.strptime(date, '%b %d, %Y')
    except:
        infos['released_date'] = ''
    
    try:
        all_plats = soup.find('div', {'class': 'platforms-info'}).findAll('title')
        platforms = []
        for plat in all_plats:
            platforms.append(plat.getText())
        infos['platform'] = platforms
    except:
        infos['platform'] = ''
    
    try:
        all_genres = soup.find('div', {'class': 'genres-info'}).findAll('a')
        genres = []
        for gens in all_genres:
            genres.append(gens.getText())
        infos['genres'] = genres
    except:
        infos['genres'] = ''
    
    try:
        infos['score'] = float(soup.find('figure', {'class': 'review-score'}).find('figcaption').getText())
    except:
        infos['score'] = ''
    
    try:
        infos['score_text'] = soup.find('div', {'class': 'score-text'}).getText()
    except:
        infos['score_text'] = ''
        
    try:
        infos['esrb'] = re.sub('ESRB: ', '', soup.find('svg', {'class': 'jsx-2211565535'}).getText())
    except:
        infos['esrb'] = ''
        
    try:
        infos['esrb_info'] = soup.findAll('div', {'class': 'content-rating-info'})[1].getText().split(', ')
    except:
        infos['esrb_info'] = ''

    games.append(infos)

In [202]:
games

[{'game': 'Magic & Mayhem',
  'developers': ['Mythos Games'],
  'publishers': ['Bethesda  Softworks'],
  'franchises': '',
  'features': ['Memory Card', 'Online'],
  'released_date': datetime.datetime(1999, 5, 15, 0, 0),
  'platform': ['PC'],
  'genres': ['Strategy'],
  'score': 7.3,
  'score_text': 'good',
  'esrb': 'Teen',
  'esrb_info': ['Animated Violence']},
 {'game': 'Fishing Master [2007]',
  'developers': ['Hudson Soft'],
  'publishers': ['Konami', 'Hudson Soft'],
  'franchises': '',
  'features': ['Number Of Players', 'Local - VS # of Players'],
  'released_date': datetime.datetime(2006, 3, 29, 0, 0),
  'platform': ['Wii'],
  'genres': ['Hunting'],
  'score': 5.5,
  'score_text': 'mediocre',
  'esrb': 'Everyone',
  'esrb_info': ['Mild Language']},
 {'game': 'Far Cry 2',
  'developers': ['Gameloft', 'Ubisoft Montreal'],
  'publishers': ['Ubisoft', 'Gameloft'],
  'franchises': ['Far Cry'],
  'features': ['Memory Card',
   'Vibration',
   'DualShock',
   'Memory Blocks',
   'Dolb

In [26]:
compi = []

for i in range(0, len(dados)):
    
    infos = {**dados.loc[i]}

    url = dados.loc[i].review_link
    response = urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        infos['game'] = soup.find('a', {'class': 'jsx-957202555'}).getText()
    except:
        infos['game'] = ''
    
    try:
        infos['score_text'] = soup.find('div', {'class': 'score-text'}).getText()
    except:
        infos['score_text'] = ''

    try:
        verdict = soup.findAll('div', {'class': 'jsx-3103488995'})[1].getText()
        infos['verdict'] = verdict
    except:
        infos['verdict'] = ''
        
    try:
        platforms = soup.findAll('a', {'class': 'jsx-3880677412'})
        platforms_list = []
        for plat in platforms:
            platforms_list.append(plat.getText())
        infos['platforms'] = platforms_list
    except:
        infos['platforms'] = ''

    reviewed_on = soup.find('figcaption', {'class': 'jsx-4081673794'}).findChildren("div")[-1].getText()
    reviewed_on = re.sub('Reviewed on ', '', reviewed_on)
    if 'Reviewed on ' in soup.find('figcaption', {'class': 'jsx-4081673794'}).findChildren("div")[-1].getText():
        infos['reviewed_on'] = reviewed_on
    else:
        infos['reviewed_on'] = ''
    
    
    if "CHOICE" in soup.find('div', {'class': 'review-score-section'}).getText():
        infos['editors_choice'] = 'yes'
    else:
        infos['editors_choice'] = 'no'
        
    try:
        date = soup.find('time', {'class': 'jsx-134961162'}).getText()
        infos['released_date'] = datetime.strptime(date, '%b %d, %Y')
    except:
        infos['released_date'] = ''
            
    
    try:
        author = soup.find('a', {'class': 'article-author'}).getText()
        infos['author'] = author
    except:
        infos['author'] = ''

            
    compi.append(infos)

In [58]:
finalList = list(setTest)
finalList

[1, 2, 3, 4, 5, 6]

In [68]:
teste1

[1, 2, 3, 4]

In [149]:
print(len(safeList), len(current_list))

9516 3987


In [150]:
linksSet = set([*safeList, *current_list])
len(linksSet)

11365

In [151]:
#PS5, SeriesX, PS4, XOne, Switch, WiiU, Wii, PS3, XBox 360, 3DS
#New 3DS, DS, Vita, PSP, iPhone, iPad, Android, XBox, Game Boy, Nintendo 64, Mac

safeList = list(linksSet)

In [152]:
len(safeList)

11365

In [154]:
dataF = pd.DataFrame(safeList)
dataF.head()

,0
0,https://www.ign.com/games/magic-mayhem
1,https://www.ign.com/games/fishing-master-2007
2,https://www.ign.com/games/far-cry-2
3,https://www.ign.com/games/pro-18-world-tour-golf
4,https://www.ign.com/games/bust-a-groove


In [175]:
dataF.to_csv('links')

In [174]:
dataF.head()

,0
0,https://www.ign.com/games/magic-mayhem
1,https://www.ign.com/games/fishing-master-2007
2,https://www.ign.com/games/far-cry-2
3,https://www.ign.com/games/pro-18-world-tour-golf
4,https://www.ign.com/games/bust-a-groove


In [172]:
dataF.columns = ['0']